# EMNIST Handwritten Character Recognizer

This notebook builds and trains a CNN model on the EMNIST Balanced dataset (digits + letters) and includes a drawing pad using OpenCV for live character prediction.

In [7]:
!pip install tensorflow tensorflow_datasets opencv-python matplotlib --quiet


In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import cv2


In [9]:
# Load EMNIST Balanced dataset (47 classes: digits + uppercase + lowercase letters)
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/balanced',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Normalize and batch the data
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, -1)  # Add channel dim
    return image, label

ds_train = ds_train.map(preprocess).cache().shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(64).prefetch(tf.data.AUTOTUNE)


In [10]:
emnist_classes = list("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabdefghnqrt")
def decode_label(index):
    return emnist_classes[index]


In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(47, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 47)             │         6,063 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 229,807 (897.68 KB)

 Trainable params: 229,807 (897.68 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(ds_train, validation_data=ds_test, epochs=10)


Epoch 1/10
   1/1763 ━━━━━━━━━━━━━━━━━━━━ 22:09 754ms/step - accuracy: 0.0000e+00 - loss: 3.8570

2025-04-24 02:16:46.824475: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


1763/1763 ━━━━━━━━━━━━━━━━━━━━ 36s 20ms/step - accuracy: 0.6661 - loss: 1.1613 - val_accuracy: 0.8407 - val_loss: 0.4747
Epoch 2/10
1763/1763 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.8513 - loss: 0.4352 - val_accuracy: 0.8615 - val_loss: 0.4084
Epoch 3/10
1763/1763 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.8728 - loss: 0.3599 - val_accuracy: 0.8655 - val_loss: 0.3954
Epoch 4/10
1763/1763 ━━━━━━━━━━━━━━━━━━━━ 36s 20ms/step - accuracy: 0.8824 - loss: 0.3218 - val_accuracy: 0.8722 - val_loss: 0.3748
Epoch 5/10
1763/1763 ━━━━━━━━━━━━━━━━━━━━ 37s 21ms/step - accuracy: 0.8933 - loss: 0.2899 - val_accuracy: 0.8719 - val_loss: 0.3781
Epoch 6/10
1763/1763 ━━━━━━━━━━━━━━━━━━━━ 36s 20ms/step - accuracy: 0.8992 - loss: 0.2648 - val_accuracy: 0.8741 - val_loss: 0.3745
Epoch 7/10
1763/1763 ━━━━━━━━━━━━━━━━━━━━ 37s 21ms/step - accuracy: 0.9061 - loss: 0.2436 - val_accuracy: 0.8734 - val_loss: 0.3908
Epoch 8/10
1763/1763 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.9102 - loss: 0.22

In [13]:
test_loss, test_acc = model.evaluate(ds_test)
print(f"Test accuracy: {test_acc:.2f}")


294/294 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8705 - loss: 0.4320
Test accuracy: 0.87


In [14]:
canvas = np.zeros((280, 280), dtype=np.uint8)

def draw_digit():
    global canvas
    window_name = "Draw a character (press 's' to submit, ESC to exit)"
    drawing = False
    last_point = None

    def draw(event, x, y, flags, param):
        nonlocal drawing, last_point
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            last_point = (x, y)
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            if last_point is not None:
                cv2.line(canvas, last_point, (x, y), 255, 20)
                last_point = (x, y)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            last_point = None

    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, draw)

    import cv2
import numpy as np

canvas = np.zeros((280, 280), dtype=np.uint8)

def draw_digit():
    global canvas
    window_name = "Draw a character (ESC to exit, Enter to predict)"
    drawing = False
    last_point = None

    def draw(event, x, y, flags, param):
        nonlocal drawing, last_point
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            last_point = (x, y)
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            if last_point is not None:
                cv2.line(canvas, last_point, (x, y), 255, 20)
                last_point = (x, y)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            last_point = None

    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, draw)

    while True:
        cv2.imshow(window_name, canvas)
        key = cv2.waitKey(1) & 0xFF

        if key == 27:  # ESC key
            break
        elif key == 13:  # ENTER key
            img = cv2.resize(canvas, (28, 28))
            img = 255 - img  # Invert
            img = img.astype("float32") / 255.0
            img = np.expand_dims(img, axis=(0, -1))

            prediction = model.predict(img)
            predicted_label = np.argmax(prediction)
            character = decode_label(predicted_label)
            print(f"🧠 Predicted Character: {character}")
            break

    cv2.destroyAllWindows()
    canvas[:] = 0  # Reset canvas

    

draw_digit()


2025-04-24 02:22:41.867 python[2128:7187062] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-24 02:22:41.867 python[2128:7187062] +[IMKInputSession subclass]: chose IMKInputSession_Modern


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🧠 Predicted Character: g
